Using a research article (by John S. Delaney - J. Chem. Inf. Comput. Sci. 2004, 44, 3, 1000-1005.) by applying Linear Regression to predict the solubility of molecules (i.e. solubility of drugs is an important physicochemical property in Drug discovery, design and development).

In [57]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [23]:
# Importing the dataset
data = pd.read_csv("../../ML_datasets/Delaney_datasets/Delaney_dataset.txt")

In [29]:
# Converting molecules from SMILES String to an rdkit object
mol = [Chem.MolFromSmiles(i) for i in data.SMILES]

In [40]:
# Calculate molecular descriptors
descrp_col_names = ["MOLWt", "MolLogP", "NumRotBonds", "AromaticProp"]
descrp_MolWt = [Descriptors.MolWt(i) for i in mol]
descrp_MolLogP = [Descriptors.MolLogP(i) for i in mol]
descrp_NumRotBonds = [Descriptors.NumRotatableBonds(i) for i in mol]
aromatic_atoms = [[i.GetAtomWithIdx(j).GetIsAromatic() for j in range(i.GetNumAtoms())] for i in mol]
sum_aromatic_atoms = [sum(i) for i in aromatic_atoms]
heavy_atoms = [Descriptors.HeavyAtomCount(i) for i in mol]
descrp_AromaticProp = [i/j for i,j in zip(sum_aromatic_atoms, heavy_atoms)]
# creating a pandas dataframe of descriptors
descrp_df = pd.DataFrame(np.array([descrp_MolWt, descrp_MolLogP, descrp_NumRotBonds, descrp_AromaticProp]).T, columns = descrp_col_names)
descrp_df.head()

,MOLWt,MolLogP,NumRotBonds,AromaticProp
0,167.850,2.5954,0.0,0.0
1,133.405,2.3765,0.0,0.0
2,167.850,2.5938,1.0,0.0
3,133.405,2.0289,1.0,0.0
4,187.375,2.9189,1.0,0.0


In [43]:
# Assigning X and Y
X = descrp_df
Y = data.iloc[:, 1]

,MOLWt,MolLogP,NumRotBonds,AromaticProp
count,1144.000000,1144.000000,1144.000000,1144.000000
mean,204.631675,2.449133,2.173951,0.364932
std,102.620500,1.866003,2.627398,0.343305
min,16.043000,-7.571400,0.000000,0.000000
25%,122.126000,1.414900,0.000000,0.000000
50%,183.594500,2.340300,1.000000,0.375000
75%,270.715750,3.406475,3.000000,0.666667
max,780.949000,10.388600,23.000000,1.000000


In [47]:
# preprocessing the X data by applying min-max norm
X = (X - X.min())/(X.max() - X.min())
#X.describe()

# splitting the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 24)

# printing...
print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")

Train: (915, 4), (915,)
Test: (229, 4), (229,)


In [52]:
# Linear regression model with MSE and r2 scores as metrics
model = linear_model.LinearRegression()
# training the model
model.fit(X_train, y_train)

LinearRegression()

In [53]:
# predicting y_train
y_tr_pred = model.predict(X_train)
# computing MSE and r2 score
tr_mse = mean_squared_error(y_train, y_tr_pred)
tr_r2 = r2_score(y_train, y_tr_pred)
tr_mse, tr_r2

(0.985255619913687, 0.7774672387512662)

In [54]:
# predicting y_test
y_ts_pred = model.predict(X_test)
# computing MSE and r2 score
ts_mse = mean_squared_error(y_test, y_ts_pred)
ts_r2 = r2_score(y_test, y_ts_pred)
ts_mse, ts_r2

(1.1234947263991966, 0.7324200963605263)

In [55]:
model.coef_, model.intercept_

(array([ -5.4747042 , -12.85249769,   0.20891181,  -0.48929741]),
 5.618331382378262)

In [58]:
# plotting parity plots between y_test and y_ts_pred
fig = px.scatter(x = y_test, y = y_ts_pred, labels = {"x": "y_test", "y": "y_ts_pred"})
fig.update_layout(title = "Parity Plot")
fig.add_shape(type = "line", line = dict(dash = "dash"), x0 = y_test.min(), y0 = y_test.min(), x1 = y_test.max(), y1 = y_test.max())
fig.show()


In [ ]:
# Next: use other models: simple Neural Networks, RFs, XGBoost